# QuickStart


In [1]:
print("Quickstart with Pytorch")

Quickstart with Pytorch


## Working With Data

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data", train=True, download = True, transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root = "data", train=False, download = True, transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


In [13]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloaders:
    print(f"Shape of X [N,C,H,W]: {X.shape} | Type: {X.dtype}")
    print(f"Shape of y: {y.shape} | Type: {y.dtype}")
    break

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28]) | Type: torch.float32
Shape of y: torch.Size([64]) | Type: torch.int64


## Creating Models

- inherit from `nn.Module`
- define layers in __init__ function
- define how data will pass through the network in `forward` function.
- move to the accelerator to accelerate the network.

In [14]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using Device: {device}")


Using Device: mps


In [15]:
# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters


To train a model we need a Loss Function and an Optimizer

Single training Loop:
- model makes the prediction
- backpropagates the prediction error to adjust the model's parameters

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

Check model's performance against test dataset

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [25]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}/n ---------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1/n ---------------
Loss: 1.042676 [   64/60000]
Loss: 1.073318 [ 6464/60000]
Loss: 0.870907 [12864/60000]
Loss: 1.037434 [19264/60000]
Loss: 0.914660 [25664/60000]
Loss: 0.946474 [32064/60000]
Loss: 0.990349 [38464/60000]
Loss: 0.934532 [44864/60000]
Loss: 0.971401 [51264/60000]
Loss: 0.921574 [57664/60000]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.919987

Epoch 2/n ---------------
Loss: 0.952949 [   64/60000]
Loss: 1.003248 [ 6464/60000]
Loss: 0.787857 [12864/60000]
Loss: 0.970303 [19264/60000]
Loss: 0.850799 [25664/60000]
Loss: 0.876605 [32064/60000]
Loss: 0.932793 [38464/60000]
Loss: 0.884055 [44864/60000]
Loss: 0.912804 [51264/60000]
Loss: 0.871720 [57664/60000]
Test Error: 
 Accuracy: 68.9%, Avg loss: 0.865320

Epoch 3/n ---------------
Loss: 0.883245 [   64/60000]
Loss: 0.949898 [ 6464/60000]
Loss: 0.725405 [12864/60000]
Loss: 0.919511 [19264/60000]
Loss: 0.805288 [25664/60000]
Loss: 0.822940 [32064/60000]
Loss: 0.888943 [38464/60000]
Loss: 0.848641 [44864/60000]
Loss: 0.

## Saving the Model

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model State to  model.pth")

Saved Pytorch Model State to  model.pth


## Loading Models

In [28]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [34]:
classes = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", 
    "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
